In [1]:
# add_special_tokens = True, 안했는데 괜찮나?

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [62.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [4]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [5]:
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip3 install /tmp/mecab-python-0.996

Installing automake (A dependency for mecab-ko)
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.l

debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 5.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1381k  100 1381k    0     0  6397k      0 --:--:-- --:--:-- --:--:-- 6397k
./configure: line 7378: /usr/bin/file: No such file or directory
In file included from viterbi.cpp:14:0:
param.h:30:13: warning: 'Target {anonymous}::lexical_cast(Source) [with Target = std::__cxx11::basic_string<char>; Source = std::_

In [6]:
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import konlpy

In [7]:
data_path = './gdrive/MyDrive/news_topic/data/'

train_origin = pd.read_csv(data_path+'train_data.csv')
test_origin = pd.read_csv(data_path+'test_data.csv')
topic_dict_origin = pd.read_csv(data_path+'topic_dict.csv')
sample_submission_origin = pd.read_csv(data_path+'sample_submission.csv')

In [8]:
train = train_origin.copy()
test = test_origin.copy()
sample_submission = sample_submission_origin.copy()

In [9]:
topic_dict = dict(zip(topic_dict_origin['topic_idx'], topic_dict_origin['topic']))

In [10]:
stopwords_origin = open(data_path+'stopwords.txt')
stopwords = stopwords_origin.read()
stopwords = stopwords.split()
stopwords_origin.close()

In [11]:
### preprocess

def text_cleaning(text, only_korean=False):
    if only_korean:
        text_rm_special_char = re.sub('[^가-힣]', ' ', text)
    else:
        text_rm_special_char = re.sub('[^a-z가-힣]', ' ', text.lower())
    text_cleaned = re.sub('[\s]+', ' ', text_rm_special_char)
    return text_cleaned

mecab = konlpy.tag.Mecab()

def remove_stopwords(text_stemmed):
    after_remove = []
    for stem in text_stemmed:
        if stem not in stopwords:
            after_remove.append(stem)
    return after_remove

def preprocess(text, only_korean=False, return_noun=False):
    text_cleaned = text_cleaning(text, only_korean)
    if return_noun:
        text_stemmed = mecab.nouns(text_cleaned)
    else:
        text_stemmed = mecab.morphs(text_cleaned)
    after_remove = remove_stopwords(text_stemmed)
    return after_remove

In [12]:
text = train['title'][0]

text_cleaned = text_cleaning(text)
print(text)
print(text_cleaned, '\n')

print(mecab.nouns(text))
print(mecab.morphs(text), '\n')

print(mecab.nouns(text_cleaned))
print(mecab.morphs(text_cleaned))

인천→핀란드 항공기 결항…휴가철 여행객 분통
인천 핀란드 항공기 결항 휴가철 여행객 분통 

['인천', '핀란드', '항공기', '결항', '휴가철', '여행객', '분통']
['인천', '→', '핀란드', '항공기', '결항', '…', '휴가철', '여행객', '분통'] 

['인천', '핀란드', '항공기', '결항', '휴가철', '여행객', '분통']
['인천', '핀란드', '항공기', '결항', '휴가철', '여행객', '분통']


In [18]:
preprocess_custom = lambda text: preprocess(text, only_korean=True, return_noun=True)
train['stem_noun'] = train['title'].apply(preprocess_custom)
test['stem_noun'] = test['title'].apply(preprocess_custom)

In [14]:
vocabulary_dict = {}

for stems, topic_idx in zip(train['stem_noun'], train['topic_idx']):
    for stem in stems:
        if stem in vocabulary_dict.keys():
            vocabulary_dict[stem]['total_count'] += 1
            vocabulary_dict[stem]['count_by_idx'][topic_dict[topic_idx]] += 1
        else:
            vocabulary_dict[stem] = {}
            vocabulary_dict[stem]['total_count'] = 1
            vocabulary_dict[stem]['count_by_idx'] = dict(zip(topic_dict.values(), [0]*7))
            vocabulary_dict[stem]['count_by_idx'][topic_dict[topic_idx]] += 1

In [15]:
vocab_freq = sorted(vocabulary_dict.keys(), key=lambda stem: vocabulary_dict[stem]['total_count'], reverse=True)
for word in vocab_freq[:20]:
    print(f'{word}: {vocabulary_dict[word]}\n')

종합: {'total_count': 4051, 'count_by_idx': {'IT과학': 329, '경제': 654, '사회': 548, '생활문화': 151, '세계': 994, '스포츠': 301, '정치': 1074}}

원: {'total_count': 2068, 'count_by_idx': {'IT과학': 189, '경제': 1184, '사회': 299, '생활문화': 71, '세계': 145, '스포츠': 136, '정치': 44}}

만: {'total_count': 1924, 'count_by_idx': {'IT과학': 300, '경제': 432, '사회': 334, '생활문화': 172, '세계': 289, '스포츠': 295, '정치': 102}}

년: {'total_count': 1833, 'count_by_idx': {'IT과학': 155, '경제': 322, '사회': 297, '생활문화': 255, '세계': 310, '스포츠': 368, '정치': 126}}

대통령: {'total_count': 1724, 'count_by_idx': {'IT과학': 2, '경제': 1, '사회': 21, '생활문화': 4, '세계': 270, '스포츠': 15, '정치': 1411}}

한국: {'total_count': 1415, 'count_by_idx': {'IT과학': 133, '경제': 167, '사회': 242, '생활문화': 246, '세계': 100, '스포츠': 260, '정치': 267}}

억: {'total_count': 1296, 'count_by_idx': {'IT과학': 66, '경제': 870, '사회': 130, '생활문화': 33, '세계': 71, '스포츠': 107, '정치': 19}}

명: {'total_count': 1282, 'count_by_idx': {'IT과학': 82, '경제': 90, '사회': 396, '생활문화': 92, '세계': 479, '스포츠': 75, '정치': 68}}

위: {

In [16]:
train

,index,title,topic_idx,stem_noun
0,0,인천→핀란드 항공기 결항…휴가철 여행객 분통,4,"[인천, 핀란드, 항공기, 결항, 휴가철, 여행객, 분통]"
1,1,실리콘밸리 넘어서겠다…구글 15조원 들여 美전역 거점화,4,"[실리콘밸리, 구글, 조원, 전역, 거점]"
2,2,이란 외무 긴장완화 해결책은 미국이 경제전쟁 멈추는 것,4,"[이란, 외무, 긴장, 완화, 해결책, 미국, 경제, 전쟁]"
3,3,NYT 클린턴 측근韓기업 특수관계 조명…공과 사 맞물려종합,4,"[클린턴, 측근, 기업, 특수, 관계, 조명, 공과, 종합]"
4,4,시진핑 트럼프에 중미 무역협상 조속 타결 희망,4,"[시진핑, 트럼프, 중미, 무역, 협상, 조속, 타결, 희망]"
...,...,...,...,...
45649,45649,KB금융 미국 IB 스티펠과 제휴…선진국 시장 공략,1,"[금융, 미국, 스티펠, 제휴, 선진국, 시장, 공략]"
45650,45650,1보 서울시교육청 신종코로나 확산에 개학 연기·휴업 검토,2,"[서울시, 교육청, 신종, 코로나, 확산, 개학, 연기, 휴업, 검토]"
45651,45651,게시판 키움증권 2020 키움 영웅전 실전투자대회,1,"[게시판, 키움증권, 영웅전, 실전, 투자, 대회]"
45652,45652,답변하는 배기동 국립중앙박물관장,2,"[답변, 배기동, 국립, 중앙, 박물, 관장]"


In [19]:
train.to_csv(data_path+'train_preprocessed.csv', index=False)
test.to_csv(data_path+'test_preprocessed.csv', index=False)